In [ ]:
%%capture
!pip install qdrant-client llama-index==0.10.25 llama-index-embeddings-openai  llama-index-vector-stores-qdrant llama-index-llms-openai

In [ ]:
import os
import sys
from getpass import getpass
import nest_asyncio

from IPython.display import Markdown, display

from dotenv import load_dotenv

nest_asyncio.apply()

load_dotenv("../.env")

sys.path.append('../helpers')

from utils import setup_llm, setup_embed_model, setup_vector_store

In [ ]:
OPENAI_API_KEY = os.environ['OPENAI_API_KEY'] or getpass("Enter your OpenAI API key: ")

In [ ]:
QDRANT_URL = os.environ['QDRANT_URL'] or getpass("Enter your Qdrant URL:")

In [ ]:
QDRANT_API_KEY = os.environ['QDRANT_API_KEY'] or  getpass("Enter your Qdrant API Key:")

In [ ]:
from llama_index.core.settings import Settings
from llama_index.llms.openai import OpenAI
from utils import setup_llm, setup_embed_model

setup_llm(provider="openai", model="gpt-3.5-turbo", api_key=OPENAI_API_KEY)

setup_embed_model(provider="openai", api_key=OPENAI_API_KEY)

In [ ]:
from datasets import load_dataset

eval_dataset = load_dataset("harpreetsahota/LI_Learning_RAG_Eval_Set", split='train')

eval_dataset = eval_dataset.filter(lambda x: x['question_groundedness_score'] is not None and x['question_groundedness_score'] >= 5)

In [ ]:
from llama_index.core.settings import Settings
from utils import setup_llm, setup_embed_model

setup_llm(
    provider="openai",
    api_key=OPENAI_API_KEY, 
    model="gpt-3.5-turbo", 
    temperature=0.75, 
    system_prompt="""Use ONLY the provided context and generate a complete, coherent answer to the user's query. 
    Your response must be grounded in the provided context and relevant to the essence of the user's query.
    """
    )

setup_embed_model(provider="openai", api_key=OPENAI_API_KEY)

In [ ]:
from llama_index.core import StorageContext
from llama_index.core.settings import Settings

from utils import create_index, create_query_engine, ingest, setup_vector_store

COLLECTION_NAME = "words-of-the-senpai-semantic-nodes"

semantic_nodes_vector_store = setup_vector_store(QDRANT_URL, QDRANT_API_KEY, COLLECTION_NAME)

semantic_nodes_storage_context = StorageContext.from_defaults(vector_store=semantic_nodes_vector_store)

semantic_nodes_index = create_index(from_where="vector_store", vector_store=semantic_nodes_vector_store, storage_context=semantic_nodes_storage_context)

# Self-Correcting Query Enginers

Self-correcting query engines in LlamaIndex evaluate their own output and then self-correct to provide better responses. They are designed to improve the quality of responses from a base query engine.

There are a few types of self-correcting query engines:

- **Retry Query Engine:** This engine uses an evaluator to improve the response from a base query engine. It first queries the base query engine, then uses the evaluator to decide if the response passes. If the response passes, it returns the response. Otherwise, it transforms the original query with the evaluation result into a new query and repeats the process up to a maximum number of retries.

- **Retry Source Query Engine:** This engine modifies the query source nodes by filtering the existing source nodes for the query based on LLM node evaluation.

- **Retry Guideline Query Engine:** This engine uses guidelines to direct the evaluator's behavior. It can be customized with your own guidelines. The engine evaluates the response against the guidelines, and if the response doesn't meet the guidelines, it transforms the query and retries.


# ↪️ [RetryQueryEngine](https://github.com/run-llama/llama_index/blob/main/llama-index-core/llama_index/core/query_engine/retry_query_engine.py)

